In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import gym
import matplotlib.pyplot as plt

In [2]:
# 相关超参数
BATCH_SIZE = 32
LR = 0.01
EPSILON = 0.9
GAMMA = 0.9
TARGET_REPLACE_ITER = 100
MEMORY_CAPACITY = 2000
env = gym.make('CartPole-v0').unwrapped
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]

In [3]:
# 定义Net
class Net(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        self.fc1 = tf.keras.layers.Dense(units=50,
                                        kernel_initializer=tf.keras.initializers.random_normal(mean=0.0,stddev=0.01),
                                        input_shape=(N_STATES,), activation='relu')
        self.fc2 = tf.keras.layers.Dense(units=N_ACTIONS,
                                        kernel_initializer=tf.keras.initializers.random_normal(mean=0.0,stddev=0.01))
    def call(self, state):
        y = self.fc1(state)
        actions_value = self.fc2(y)
        return actions_value

In [6]:
# 定义DQN类
class Agent_DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()
        self.step = 0
        self.memory_counter = 0
        self.memory = np.zeros(shape=(MEMORY_CAPACITY, N_STATES * 2 + 2))
        self.optimizer = tf.keras.optimizers.Adam((LR))
        self.loss = tf.keras.losses.MSE
        self.eval_net.compile(optimizer=self.optimizer,
                         loss=self.loss)
        self.target_net.compile(optimizer=self.optimizer,
                         loss=self.loss)
        
        self.target_net(tf.ones((1,N_STATES)))
        self.eval_net(tf.ones((1,N_STATES)))
        self.eval_net.set_weights(self.target_net.get_weights())
       
        
    
    def choice_action(self, state):
        '''
        state: array([1,2,3,4])
        '''
        # 如果小于epsilon则选择最优的策略
        state = np.expand_dims(state, axis=0)
        if np.random.rand() < EPSILON:
            return np.argmax(np.array(self.eval_net(state)))
        else:
            return np.random.randint(2)
    
    def store_transition(self, state, action, reward, next_state):
        transition = np.hstack([state, action, reward, next_state])
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index] = transition
        self.memory_counter += 1
    
    def learn(self):
        if self.step % 100 == 0:
            self.target_net.set_weights(self.eval_net.get_weights())
        self.step += 1
        # 抽取数据
        index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[index, : ]
        b_state = b_memory[:, :N_STATES]
        b_action = b_memory[:, N_STATES: N_STATES+1]
        b_reward = b_memory[:, N_STATES+1: N_STATES+2]
        b_next_state = b_memory[:, N_STATES+2:]
        
        q_target = self.target_net(b_next_state)
        U = tf.constant(b_reward, dtype=tf.float32) + GAMMA * tf.reduce_max(q_target, axis=1, keepdims=True)
        self.eval_net.fit(b_state, U)

In [ ]:
Agent = Agent_DQN()
episode_rewards = []
for i in range(500):
    if i % 10 == 0:
        print('<<<<<<<<<Episode: %s' % i)
    # 重置环境
    state = env.reset()                                                     
    # 初始化该循环对应的episode的总奖励
    episode_reward_sum = 0                                              

    while True:
        #env.render()
        action = Agent.choice_action(state)
        next_state, reward, done, _ = env.step(action)
        
        # 修改奖励 (不修改也可以，修改奖励只是为了更快地得到训练好的摆杆)
        x, x_dot, theta, theta_dot = next_state
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        new_reward = r1 + r2
        # 存储样本
        Agent.store_transition(state, action, reward, next_state)
        # 逐步加上一个episode内每个step的reward
        episode_reward_sum += reward
        # 更新状态
        state = next_state                                                
        # 如果累计的transition数量超过了记忆库的固定容量2000
        if Agent.memory_counter > 2000:
            # 开始学习 (抽取记忆，即32个transition，并对评估网络参数进行更新，并在开始学习后每隔100次将评估网络的参数赋给目标网络)
            Agent.learn()
        if done:       # 如果done为True
            # round()方法返回episode_reward_sum的小数点四舍五入到2个数字
            print('episode%s---reward_sum: %s' % (i, round(episode_reward_sum, 2)))
            episode_rewards.append(episode_reward_sum)
            break                                             # 该episode结束

<<<<<<<<<Episode: 0
episode0---reward_sum: 9.0
episode1---reward_sum: 8.0
episode2---reward_sum: 10.0
episode3---reward_sum: 9.0
episode4---reward_sum: 8.0
episode5---reward_sum: 10.0
episode6---reward_sum: 8.0
episode7---reward_sum: 9.0
episode8---reward_sum: 9.0
episode9---reward_sum: 9.0
<<<<<<<<<Episode: 10
episode10---reward_sum: 9.0
episode11---reward_sum: 10.0
episode12---reward_sum: 18.0
episode13---reward_sum: 16.0
episode14---reward_sum: 8.0
episode15---reward_sum: 11.0
episode16---reward_sum: 9.0
episode17---reward_sum: 10.0
episode18---reward_sum: 9.0
episode19---reward_sum: 12.0
<<<<<<<<<Episode: 20
episode20---reward_sum: 9.0
episode21---reward_sum: 8.0
episode22---reward_sum: 9.0
episode23---reward_sum: 9.0
episode24---reward_sum: 8.0
episode25---reward_sum: 10.0
episode26---reward_sum: 13.0
episode27---reward_sum: 9.0
episode28---reward_sum: 9.0
episode29---reward_sum: 8.0
<<<<<<<<<Episode: 30
episode30---reward_sum: 9.0
episode31---reward_sum: 9.0
episode32---reward_su

1/1 [==============================] - 0s 2ms/step - loss: 0.0063
episode190---reward_sum: 37.0
1/1 [==============================] - 0s 2ms/step - loss: 2.2093e-04
episode191---reward_sum: 27.0
1/1 [==============================] - 0s 2ms/step - loss: 1.5840e-04
episode192---reward_sum: 10.0
1/1 [==============================] - 0s 2ms/step - loss: 1.1443e-04
episode193---reward_sum: 11.0
1/1 [==============================] - 0s 2ms/step - loss: 0.3618
episode194---reward_sum: 11.0
1/1 [==============================] - 0s 3ms/step - loss: 0.0227
episode195---reward_sum: 15.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0035
episode196---reward_sum: 14.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0055
episode197---reward_sum: 14.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0012


1/1 [==============================] - 0s 2ms/step - loss: 3.0658e-04
episode198---reward_sum: 10.0
1/1 [==============================] - 0s 2ms/step - loss: 2.2846e-04
episode199---reward_sum: 16.0
<<<<<<<<<Episode: 200
1/1 [==============================] - 0s 2ms/step - loss: 2.9926e-04
episode200---reward_sum: 14.0
1/1 [==============================] - 0s 2ms/step - loss: 0.4736
episode201---reward_sum: 12.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0070
episode202---reward_sum: 9.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0519
episode203---reward_sum: 9.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0069
episode204---reward_sum: 10.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0101
episode205---reward_sum: 11.0
1/1 [==============================] - 0s 4ms/step - loss: 0.0019
episode206---reward_sum: 14.0
1/1 [==============================] - 0s 3ms/step - loss: 6.5162e-04


1/1 [==============================] - 0s 3ms/step - loss: 7.0354e-04
episode207---reward_sum: 15.0
1/1 [==============================] - 0s 2ms/step - loss: 2.4480e-04
episode208---reward_sum: 16.0
1/1 [==============================] - 0s 3ms/step - loss: 0.3496
episode209---reward_sum: 16.0
<<<<<<<<<Episode: 210
1/1 [==============================] - 0s 2ms/step - loss: 0.1691
episode210---reward_sum: 10.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0452
episode211---reward_sum: 13.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0156
episode212---reward_sum: 12.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0034
episode213---reward_sum: 11.0
1/1 [==============================] - 0s 3ms/step - loss: 0.0011
episode214---reward_sum: 13.0
1/1 [==============================] - 0s 2ms/step - loss: 7.6614e-04
episode215---reward_sum: 12.0
1/1 [==============================] - 0s 2ms/step - loss: 1.2990e-04


1/1 [==============================] - 0s 2ms/step - loss: 2.0712e-04
episode216---reward_sum: 12.0
1/1 [==============================] - 0s 2ms/step - loss: 1.5304e-04
episode217---reward_sum: 9.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0314
episode218---reward_sum: 10.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0752
episode219---reward_sum: 10.0
<<<<<<<<<Episode: 220
1/1 [==============================] - 0s 2ms/step - loss: 0.0021
episode220---reward_sum: 13.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0030
episode221---reward_sum: 16.0
1/1 [==============================] - 0s 2ms/step - loss: 4.4809e-04
episode222---reward_sum: 18.0
1/1 [==============================] - 0s 2ms/step - loss: 1.0607e-04
episode223---reward_sum: 14.0
1/1 [==============================] - 0s 2ms/step - loss: 7.7267e-05
episode224---reward_sum: 13.0
1/1 [==============================] - 0s 3ms/step - loss: 5.0514e-05


...

1/1 [==============================] - 0s 2ms/step - loss: 0.0014
episode448---reward_sum: 24.0
1/1 [==============================] - 0s 3ms/step - loss: 0.0018
episode449---reward_sum: 32.0
<<<<<<<<<Episode: 450
1/1 [==============================] - 0s 2ms/step - loss: 8.2024e-04
episode450---reward_sum: 14.0
1/1 [==============================] - 0s 2ms/step - loss: 3.4535e-04
episode451---reward_sum: 24.0
1/1 [==============================] - 0s 3ms/step - loss: 6.6215e-04
episode452---reward_sum: 32.0
1/1 [==============================] - 0s 2ms/step - loss: 4.1079e-04


1/1 [==============================] - 0s 3ms/step - loss: 5.6663e-04
episode453---reward_sum: 36.0
1/1 [==============================] - 0s 3ms/step - loss: 7.2753e-04
episode454---reward_sum: 41.0
1/1 [==============================] - 0s 2ms/step - loss: 7.9301e-04
episode455---reward_sum: 43.0
1/1 [==============================] - 0s 3ms/step - loss: 0.0032


1/1 [==============================] - 0s 3ms/step - loss: 0.0125
episode456---reward_sum: 20.0
1/1 [==============================] - 0s 3ms/step - loss: 0.0048
episode457---reward_sum: 20.0
1/1 [==============================] - 0s 3ms/step - loss: 0.0015
episode458---reward_sum: 17.0
1/1 [==============================] - 0s 3ms/step - loss: 0.0012
episode459---reward_sum: 22.0
<<<<<<<<<Episode: 460
1/1 [==============================] - 0s 3ms/step - loss: 0.0019
episode460---reward_sum: 19.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0089
episode461---reward_sum: 18.0
1/1 [==============================] - 0s 2ms/step - loss: 0.0012


1/1 [==============================] - 0s 3ms/step - loss: 0.0018
episode462---reward_sum: 13.0
1/1 [==============================] - 0s 3ms/step - loss: 8.9306e-04
episode463---reward_sum: 35.0
1/1 [==============================] - 0s 3ms/step - loss: 0.0024
episode464---reward_sum: 31.0
1/1 [==============================] - 0s 3ms/step - loss: 6.8812e-04
episode465---reward_sum: 16.0
1/1 [==============================] - 0s 3ms/step - loss: 0.0010


1/1 [==============================] - 0s 3ms/step - loss: 0.0011
episode466---reward_sum: 39.0
1/1 [==============================] - 0s 3ms/step - loss: 0.0010
episode467---reward_sum: 26.0
1/1 [==============================] - 0s 3ms/step - loss: 6.3497e-04
episode468---reward_sum: 13.0
1/1 [==============================] - 0s 3ms/step - loss: 8.3999e-04


In [ ]:
fig, axes = plt.subplots(1,1,figsize=(8,6))
axes.plot(episode_rewards)
axes.set_title('episode_rewards')